In [276]:
import pandas as pd
from ast import literal_eval
from datetime import datetime
import re

In [277]:
#using the new csv file here
csv_file = "new_movie_metadata.csv"

df = pd.read_csv(csv_file, header= 0, encoding= 'unicode_escape')


C:\Users\bvera\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3058: DtypeWarning: Columns (0,2,4,9,19) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [278]:
df.columns

Index(['adult', 'belongs_to_collection', 'budget', 'homepage', 'id', 'imdb_id',
       'original_language', 'original_title', 'overview', 'popularity',
       'poster_path', 'production_countries', 'release_date', 'revenue',
       'runtime', 'spoken_languages', 'status', 'tagline', 'title', 'video',
       'vote_average', 'vote_count', 'production_companies', 'genre'],
      dtype='object')

In [279]:
df["release_date"]

0        12/10/2009
1        12/15/2015
2        11/18/1997
3         4/25/2012
4          6/9/2015
            ...    
45602           NaN
45603           NaN
45604           NaN
45605           NaN
45606           NaN
Name: release_date, Length: 45607, dtype: object

In [280]:
release_date = df[['id','release_date']].head()

In [281]:
release_date

,id,release_date
0,19995,12/10/2009
1,140607,12/15/2015
2,597,11/18/1997
3,24428,4/25/2012
4,135397,6/9/2015


In [282]:
#a for loop to pick up dates in the excel that are messy.
year_list = []

for row_number, date in df["release_date"].items():
    if isinstance(date, float) or len(date) <=3:
        year_list.append(None)
    else:
        try:
            date_year = datetime.strptime(date, "%m/%d/%Y")
            year_list.append(date_year.year)
        except ValueError:
            date_year = datetime.strptime(date, "%Y-%m-%d")
            year_list.append(date_year.year)

In [283]:
#new column, who dis?
df["release_year"] = year_list

In [284]:
df.columns

Index(['adult', 'belongs_to_collection', 'budget', 'homepage', 'id', 'imdb_id',
       'original_language', 'original_title', 'overview', 'popularity',
       'poster_path', 'production_countries', 'release_date', 'revenue',
       'runtime', 'spoken_languages', 'status', 'tagline', 'title', 'video',
       'vote_average', 'vote_count', 'production_companies', 'genre',
       'release_year'],
      dtype='object')

In [285]:
df[["release_date","release_year"]]

,release_date,release_year
0,12/10/2009,2009.0
1,12/15/2015,2015.0
2,11/18/1997,1997.0
3,4/25/2012,2012.0
4,6/9/2015,2015.0
...,...,...
45602,NaN,NaN
45603,NaN,NaN
45604,NaN,NaN
45605,NaN,NaN


In [286]:
#testing coversion of revenue from float to currency
revenues = df[["id","revenue","budget"]].head(5)

In [287]:
revenues

,id,revenue,budget
0,19995,2.787965e+09,237000000
1,140607,2.068224e+09,245000000
2,597,1.845034e+09,200000000
3,24428,1.519558e+09,220000000
4,135397,1.513529e+09,150000000


In [288]:
#confirmed
revenues["revenue"].astype(int)
# revenues["revenue"] = revenues["revenue"].map("${:,.0f}".format)

0   -2147483648
1    2068223624
2    1845034188
3    1519557910
4    1513528810
Name: revenue, dtype: int32

In [289]:
#same for budget
revenues["budget"].astype(int)

0    237000000
1    245000000
2    200000000
3    220000000
4    150000000
Name: budget, dtype: int32

In [290]:
revenues

,id,revenue,budget
0,19995,2.787965e+09,237000000
1,140607,2.068224e+09,245000000
2,597,1.845034e+09,200000000
3,24428,1.519558e+09,220000000
4,135397,1.513529e+09,150000000


In [291]:
#revenue was once a float
df.dtypes

adult                     object
belongs_to_collection     object
budget                    object
homepage                  object
id                        object
imdb_id                   object
original_language         object
original_title            object
overview                  object
popularity                object
poster_path               object
production_countries      object
release_date              object
revenue                  float64
runtime                  float64
spoken_languages          object
status                    object
tagline                   object
title                     object
video                     object
vote_average             float64
vote_count               float64
production_companies      object
genre                     object
release_year             float64
dtype: object

In [292]:
production = df[["production_companies","revenue","budget","title","release_year"]].copy()

In [293]:
production = production.dropna(subset=['title','production_companies'])

In [294]:
top = production.sort_values(by = ["revenue"],ascending = False)

In [295]:
#top 20 films with the highest revenue. now trying to group studios by their names 
top.head(20)

,production_companies,revenue,budget,title,release_year
0,"Ingenious Film Partners, Twentieth Century Fox...",2.787965e+09,237000000,Avatar,2009.0
1,"Lucasfilm, Truenorth Productions, Bad Robot",2.068224e+09,245000000,Star Wars: The Force Awakens,2015.0
2,"Paramount Pictures, Twentieth Century Fox Film...",1.845034e+09,200000000,Titanic,1997.0
3,"Paramount Pictures, Marvel Studios",1.519558e+09,220000000,The Avengers,2012.0
4,"Universal Studios, Amblin Entertainment, Legen...",1.513529e+09,150000000,Jurassic World,2015.0
5,"Universal Pictures, Original Film, Fuji Televi...",1.506249e+09,190000000,Furious 7,2015.0
6,"Marvel Studios, Prime Focus, Revolution Sun St...",1.405404e+09,280000000,Avengers: Age of Ultron,2015.0
7,"Warner Bros., Heyday Films",1.342000e+09,125000000,Harry Potter and the Deathly Hallows: Part 2,2011.0
8,"Walt Disney Pictures, Walt Disney Animation St...",1.274219e+09,150000000,Frozen,2013.0
9,"Walt Disney Pictures, Mandeville Films",1.262886e+09,160000000,Beauty and the Beast,2017.0


In [296]:
Marvel = production[production['production_companies'].str.contains('Marvel')]

In [297]:
len(Marvel)

65

In [299]:
sort_year = Marvel.sort_values(by = ["release_year"],ascending = False)

In [337]:
#dropped team thor since it had 0 info
top_ten = sort_year.set_index('title').drop(['Team Thor','Marvel Studios: Assembling a Universe','Marvel One-Shot: All Hail the King','Iron Man & Captain America: Heroes United','Marvel: 75 Years, From Pulp to Pop!','Iron Man & Hulk: Heroes United','Marvel One-Shot: Agent Carter','Marvel One-Shot: Item 47','Thor: Tales of Asgard','Planet Hulk'])

In [338]:
top_ten.head(57)

,production_companies,revenue,budget,release_year
title,,,,
Guardians of the Galaxy Vol. 2,"Walt Disney Pictures, Marvel Studios",8.634161e+08,200000000,2017.0
Logan,"Twentieth Century Fox Film Corporation, Donner...",6.168018e+08,97000000,2017.0
Thor: Ragnarok,"Walt Disney Pictures, Marvel Studios",8.540000e+08,180000000,2017.0
Captain America: Civil War,"Studio Babelsberg, Marvel Studios, Walt Disney...",1.153304e+09,250000000,2016.0
X-Men: Apocalypse,"Twentieth Century Fox Film Corporation, Donner...",5.439348e+08,178000000,2016.0
Deadpool,"Twentieth Century Fox Film Corporation, Marvel...",7.831130e+08,58000000,2016.0
Doctor Strange,"Walt Disney Pictures, Marvel Studios",6.777184e+08,165000000,2016.0
Avengers: Age of Ultron,"Marvel Studios, Prime Focus, Revolution Sun St...",1.405404e+09,280000000,2015.0
Ant-Man,Marvel Studios,5.193120e+08,130000000,2015.0


In [339]:
len(top_ten)

55

In [340]:
top_ten.reset_index(level='title')

,title,production_companies,revenue,budget,release_year
0,Guardians of the Galaxy Vol. 2,"Walt Disney Pictures, Marvel Studios",8.634161e+08,200000000,2017.0
1,Logan,"Twentieth Century Fox Film Corporation, Donner...",6.168018e+08,97000000,2017.0
2,Thor: Ragnarok,"Walt Disney Pictures, Marvel Studios",8.540000e+08,180000000,2017.0
3,Captain America: Civil War,"Studio Babelsberg, Marvel Studios, Walt Disney...",1.153304e+09,250000000,2016.0
4,X-Men: Apocalypse,"Twentieth Century Fox Film Corporation, Donner...",5.439348e+08,178000000,2016.0
5,Deadpool,"Twentieth Century Fox Film Corporation, Marvel...",7.831130e+08,58000000,2016.0
6,Doctor Strange,"Walt Disney Pictures, Marvel Studios",6.777184e+08,165000000,2016.0
7,Avengers: Age of Ultron,"Marvel Studios, Prime Focus, Revolution Sun St...",1.405404e+09,280000000,2015.0
8,Ant-Man,Marvel Studios,5.193120e+08,130000000,2015.0
9,Fantastic Four,"Twentieth Century Fox Film Corporation, Marv F...",1.679776e+08,120000000,2015.0


In [355]:
#convert budget to float since sum below wouldnt work as a object
top_ten["budget"] = top_ten["budget"].astype(float)

In [358]:
revenue_sum = top_ten.groupby(['release_year'])['revenue'].sum()

budget_sum = top_ten.groupby(['release_year'])['budget'].sum()

In [363]:
merge_inner = pd.merge(left=revenue_sum, right=budget_sum, left_on='release_year', right_on='release_year')

In [365]:
merge_inner.sort_values(by = ["release_year"],ascending = False)

,revenue,budget
release_year,,
2017.0,2.334218e+09,477000000.0
2016.0,3.158071e+09,651000000.0
2015.0,2.092693e+09,530000000.0
2014.0,2.941675e+09,790000000.0
2013.0,2.275452e+09,490000000.0
2012.0,2.271774e+09,435000000.0
2011.0,1.322738e+09,508000000.0
2010.0,6.239333e+08,200000000.0
2009.0,3.730629e+08,150000000.0
